### Assignment 2: DES Simulation

In [2]:
import numpy as np
import simpy
import random

In [24]:

# Parameters
RANDOM_SEED = 42
SIM_TIME = 10000  # Increased simulation time
ARRIVAL_RATE = 10
SERVICE_RATE = 12
NUM_SERVERS = [1, 2, 4]

def service(env, server, service_rate):
    """Simulate the service of a job by a server."""
    service_time = random.expovariate(service_rate)
    yield env.timeout(service_time)

def process_job(env, job_id, servers, service_rate, waiting_time):
    """Handle the arrival and processing of a job."""
    arrival_time = env.now

    with servers.request() as request:
        yield request  # Wait for a server to be available
        wait_time = env.now - arrival_time
        waiting_time.append(wait_time)  # Record the waiting time for the job
        yield env.process(service(env, request, service_rate))

def arrival_process(env, arrival_rate, servers, service_rate, waiting_time):
    """Generate new jobs that arrive with an exponential interarrival time."""
    job_id = 0
    while True:
        interarrival_time = random.expovariate(arrival_rate)
        yield env.timeout(interarrival_time)
        env.process(process_job(env, job_id, servers, service_rate, waiting_time))
        job_id += 1

def run_simulation(num_servers, arrival_rate, service_rate):
    """Run the simulation."""
    random.seed(RANDOM_SEED)
    
    # Create the simulation environment
    env = simpy.Environment()

    # Create the server resource (how many servers are available)
    servers = simpy.Resource(env, capacity=num_servers)
    
    # List to store waiting times
    waiting_time = []
    
    # Start the arrival process
    env.process(arrival_process(env, arrival_rate, servers, service_rate, waiting_time))
    
    # Run the simulation for the specified time
    env.run(until=SIM_TIME)
    
    # Return the average waiting time
    return np.mean(waiting_time)

if __name__ == '__main__':
    for num_servers in NUM_SERVERS:
        avg_waiting_time = run_simulation(num_servers, ARRIVAL_RATE, SERVICE_RATE)
        print(f'Average waiting time with {num_servers} servers: {avg_waiting_time:.2f}')



Average waiting time with 1 servers: 0.43
Average waiting time with 2 servers: 0.02
Average waiting time with 4 servers: 0.00
